In [ ]:
### Load Libraries.

library("scDC")
library("Seurat")
library("dplyr")
library("ggplot2")
library("RColorBrewer")
library("tidyverse")
library("broom.mixed")
library("scran")

In [ ]:
### Load Data ---

setwd("/folder/") 
Seurat <- readRDS("adata.rds")

In [ ]:
### Prepare Metadata

Seurat@meta.data$Patient_Status <- paste0(Seurat@meta.data$Patient, "_", seurat_obj@meta.data$Status)

In [ ]:
### Order metadata by diagnostic category
status_order <- c("Control", "C9ALS", "sALS")
Seurat@meta.data <- Seurat@meta.data %>%
  mutate(Status = factor(Status, levels = status_order)) %>%
  arrange(Status)

unique(Seurat@meta.data$Patient_Status)

In [ ]:
### Extract Normalized Expression Matrix

norm_counts <- GetAssayData(Seurat, slot = "data")

cell_types <- Seurat@meta.data$Basic_Clusters
subjects <- Seurat@meta.data$Patient_Status
statuses <- Seurat@meta.data$Status

In [ ]:
### Run scDC without Clustering (Direct Proportion-Based CI Estimation)

res_scDC_noClust <- scDC_noClustering(
  cellTypes = cell_types,
  subject = subjects,
  calCI = TRUE,
  calCI_method = c("percentile", "BCa", "multinom"),
  nboot = 200
)

In [ ]:
### Visualization of Cell Type Proportions

barplotCI(res_scDC_noClust, group = statuses)
densityCI(res_scDC_noClust, group = statuses)

In [ ]:
### Differential Abundance Testing via Generalized Linear Model

res_glm <- fitGLM(
  res_scDC_noClust,
  group = statuses,
  pairwise = TRUE
)

In [ ]:
### Summaries of Fixed and Random Effects.

summary(res_glm$pool_res_fixed)
summary(res_glm$pool_res_random)

In [ ]:
### Extract and adjust p-values (FDR correction).

glm_summary <- summary(res_glm$pool_res_fixed)
glm_summary$adj_p_Bonf <- p.adjust(glm_summary$p.value, method = "bonferroni")
glm_summary$adj_p_BH <- p.adjust(glm_summary$p.value, method = "BH")

glm_summary

In [ ]:
### Run scDC with Clustering-based Correction

res_scDC_clust <- scDC_clustering(
  exprsMat = norm_counts,
  cellTypes = cell_types,
  subject = subjects,
  calCI = TRUE,
  calCI_method = c("percentile", "BCa", "multinom"),
  nboot = 50,
  verbose = TRUE
)

In [ ]:
### Visualize CI Intervals Across Groups.

barplotCI(res_scDC_clust, group = statuses)
densityCI(res_scDC_clust, group = statuses)

In [ ]:
# Test DA on Clustered Results

res_glm_clust <- fitGLM(res_scDC_clust, group = statuses, pairwise = TRUE)
summary(res_glm_clust$pool_res_fixed)